# 使用fastflow架構進行異常檢測



## 下載程式碼&訓練資料與demo權重

In [ ]:
!git clone https://github.com/gathierry/FastFlow.git
%cd FastFlow/
!pip3 install timm pyyaml pytorch-ignite 'FrEIA @ git+https://github.com/VLL-HD/FrEIA@1779d1fba1e21000fda1927b59eeac0a6fcaa284'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1A5FVE_dOspyOFZfSqyNMiL63XeMV6V0F" -O data.zip && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IElkg-76ieMlXX3Jyzx7BH_lekv480Tw' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1IElkg-76ieMlXX3Jyzx7BH_lekv480Tw" -O demo.pt && rm -rf /tmp/cookies.txt
!tar xvf data.zip

* 因為套件版本關係，需要對原程式進行些許改動才可以執行
1.  請先點擊左側資料夾，並找到FastFlow內的main.py
2.  打開它，並找到第99行，進行以下替換  
Before: targets = targets.flatten()  
After: targets = targets.flatten().type(torch.int32)  
3.  按下'ctrl+s'存檔


## 模型訓練(若使用demo權重，則不需執行此程式)

In [ ]:
!python main.py -cfg configs/resnet18.yaml --data ./data -cat transistor

準確度測試  
-ckpt 後面請接上demo.pt 或是你自己訓練的權重

In [ ]:
!python main.py -cfg configs/resnet18.yaml --data ./data -cat transistor --eval -ckpt /content/FastFlow/demo.pt

## 模型推論&並顯示熱圖

宣告要使用的套件庫


In [9]:
import argparse
import os
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import yaml
from ignite.contrib import metrics
import numpy as np
import constants as const
import dataset
import fastflow
import utils

製作熱圖

In [10]:
def create_heatmap(data):
    # 轉換數據為浮點型並歸一化到 [0, 1]
    normalized_data = (data - np.min(data)) / 2

    # 設置熱度圖顏色映射（從藍色到紅色）
    colormap = cv2.COLORMAP_JET

    # 將數據映射到熱度圖
    heatmap = cv2.applyColorMap(np.uint8(255 * normalized_data), colormap)
    return heatmap

加載測試集

In [11]:
# 設置資料路徑
data_path = './data/transistor/'

# 定義預處理轉換
transform = transforms.Compose([
    transforms.Resize((256, 256)),   # 調整圖像大小
    transforms.ToTensor(),# 轉換為張量
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
# 加載測試集
test_dataset = ImageFolder(root=data_path+'/test', transform=transform)

# 創建訓練集和測試集的數據加載器
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

建立fastflow模型


In [13]:
def build_model():
    model = fastflow.FastFlow(
        backbone_name='resnet18',
        flow_steps=8,
        input_size=256,
        conv3x3_only=1,
        hidden_ratio=1.0,
    )
    print(
        "Model A.D. Param#: {}".format(
            sum(p.numel() for p in model.parameters() if p.requires_grad)
        )
    )
    return model

載入fastflow模型權重

In [ ]:
checkpoint= '/content/FastFlow/demo.pt'
model = build_model()
checkpoint = torch.load(checkpoint)
model.load_state_dict(checkpoint["model_state_dict"])
model.cuda()

使用測試集進行測試，並製作熱圖

In [ ]:
from google.colab.patches import cv2_imshow
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
model.eval()
for data, targets in test_loader:
    data, targets = data.cuda(), targets.cuda()
    with torch.no_grad():
        ret = model(data)
    outputs = ret["anomaly_map"].cpu().detach()
    images = np.transpose(data[0].cpu().numpy(), (1, 2, 0))*std+mean
    images = cv2.cvtColor(np.array(images*255,dtype=np.uint8), cv2.COLOR_RGB2BGR)
    outputs = np.transpose(outputs[0].numpy(), (1, 2, 0))
    heatmap = create_heatmap(outputs)
    print(np.max(outputs))
    img = np.array(images*0.8+heatmap*0.2)
    cv2_imshow(cv2.resize(np.array(img,dtype=np.uint8),(400,400)))
